In [11]:
# !pip install -q langchain
# !pip install -q openai

In [7]:
from keys import openapi_key
import os

os.environ['OPENAI_API_KEY'] = openapi_key


In [9]:
from langchain.llms import OpenAI
llm = OpenAI(temperature=0.6)
name = llm("I want to open a restaurant for Chinese food. Suggest a fancy name for this.")
print(name)



"Imperial Dragon Palace"


In [10]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables = ["cuisine"],
    template="I want to open a restaurant for {cuisine} food. Suggest a fancy name for this.",
)

prompt_template_name.format(cuisine="Mexican")

'I want to open a restaurant for Mexican food. Suggest a fancy name for this.'

In [15]:
from langchain.chains import LLMChain

name_chain = LLMChain(llm=llm, prompt=prompt_template_name)
name_chain.run(cuisine="Mexican")

'\n\n"La Cantina de Oro"'

## Sequential Chains

In [19]:
prompt_template_items = PromptTemplate(
    input_variables = ["restaurant_name"],
    template="Suggest some menu items for {restaurant_name}. Return it as a comma separated list.",
)
food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

In [20]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains=[name_chain, food_items_chain])
response = chain.run("Indian")
print(response)



1. Spicy Chicken Curry
2. Lamb Vindaloo
3. Vegetable Samosas
4. Tandoori Shrimp
5. Naan Bread
6. Chicken Tikka Masala
7. Aloo Gobi (Potato and Cauliflower Curry)
8. Palak Paneer (Spinach and Cheese Curry)
9. Mango Lassi
10. Biryani Rice
11. Chana Masala (Chickpea Curry)
12. Chicken Korma
13. Garlic Naan
14. Masala Dosa (Stuffed Crepe)
15. Gulab Jamun (Sweet Fried Dumplings)


## Sequential Chain

In [24]:
llm = OpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables=["cuisine"],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fancy name for this.",
)

name_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

prompt_template_items = PromptTemplate(
    input_variables=["restaurant_name"],
    template = "Suggest some menu items for {restaurant_name}.",
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")



In [25]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains=[name_chain, food_items_chain], 
    input_variables=["cuisine"], 
    output_variables = ["restaurant_name", "menu_items"]
)

chain({"cuisine": "Singaporean"})

{'cuisine': 'Singaporean',
 'restaurant_name': '\n\n"Singapore Spice House"',
 'menu_items': '\n\n1. Singaporean Chili Crab\n2. Hainanese Chicken Rice\n3. Laksa Soup\n4. Char Kway Teow (Stir-fried noodles with shrimp, egg, and sausage)\n5. Satay Skewers (grilled marinated meat on skewers)\n6. Nasi Lemak (coconut rice dish with fried chicken, fried anchovies, peanuts, and sambal chili sauce)\n7. Roti Prata (Indian-style flatbread served with curry)\n8. Bak Kut Teh (pork rib soup with herbs and spices)\n9. Murtabak (stuffed pancake with chicken or mutton)\n10. Singaporean-style Fried Rice\n11. Beef Rendang (spicy braised beef in coconut milk)\n12. Chilli Prawns\n13. Gado Gado (Indonesian salad with peanut sauce)\n14. Sambal Stingray (grilled stingray with spicy sambal sauce)\n15. Biryani Rice (fragrant rice dish with spices and choice of meat)'}